In [1]:
import boto3
import data_collection
import data_storage
import json
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8')

bucket = "seatgeek-tickets"

In [2]:
data = data_collection.etl()
# resp = data_storage.put_object(data, bucket)


(1/17) https://seatgeek.com/nets-at-76ers-tickets/1-25-2023-philadelphia-pennsylvania-wells-fargo-center/nba/5775973
game=Brooklyn-Nets data-size=245


ValueError: too many values to unpack (expected 2)

In [ ]:
# master_df = pd.concat(dfs)
# master_df['section'] = master_df['section'].apply(lambda x: x.split(',')[0])

# section_size = master_df.groupby('section').size()
# valid_sections = section_size.index.values[section_size > 3]
# valid_sections

In [24]:
test['Boston Celtics(1)']

'{"price":{"0":"$584 each","1":"$674 each","2":"$426 each","3":"$779 each","4":"$62 each","5":"$494 each","6":"$100 each","7":"$443 each","8":"$494 each","9":"$84 each","10":"$71 each","11":"$88 each","12":"$473 each","13":"$69 each","14":"$76 each","15":"$492 each","16":"$426 each","17":"$345 each","18":"$269 each","19":"$494 each","20":"$390 each","21":"$76 each","22":"$84 each","23":"$82 each","24":"$88 each","25":"$570 each","26":"$86 each","27":"$87 each","28":"$63 each","29":"$84 each","30":"$84 each","31":"$73 each","32":"$91 each","33":"$471 each","34":"$69 each","35":"$494 each","36":"$63 each","37":"$667 each","38":"$73 each","39":"$231 each","40":"$71 each","41":"$167 each","42":"$62 each","43":"$443 each","44":"$72 each","45":"$99 each","46":"$150 each","47":"$68 each","48":"$88 each"},"score":{"0":"10.0","1":"10.0","2":"9.9","3":"9.9","4":"9.9","5":"9.9","6":"9.9","7":"9.9","8":"9.9","9":"9.9","10":"9.8","11":"9.8","12":"9.8","13":"9.8","14":"9.8","15":"9.8","16":"9.8","17